<a href="https://colab.research.google.com/github/seoyeon0905/mimic-readmission-analysis/blob/main/notebooks/02_cohort_definition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Colab은 실행 환경이 매번 새로 시작될 수 있어서,
# GitHub 레포의 data/ 폴더와 노트북 파일을 로컬(/content)로 가져와야 함.
# 즉, "GitHub에 저장된 파일을 Colab 런타임으로 복사"하는 과정.

!git clone https://github.com/seoyeon0905/mimic-readmission-analysis.git

fatal: destination path 'mimic-readmission-analysis' already exists and is not an empty directory.


# 02. Cohort Definition and Readmission Label Construction

## What this notebook does
- This notebook **defines the study cohort** and **creates a 30-day readmission label**.
- Target cohort: **patients with a first hospital admission that includes an ICU stay**.
- Target label: whether the patient is **readmitted within 30 days** after discharge.

## Why this notebook matters (portfolio/interview point)
In clinical data analysis, you cannot start modelling immediately.
You must first:
1) decide **who is included** (cohort definition), and  
2) define the outcome **in data terms** (label construction).

This notebook demonstrates:
- understanding of **relational clinical tables**
- correct handling of **time-based outcomes**
- ability to build a valid dataset for EDA/modelling

---

# 02. 코호트 정의 및 재입원 라벨 생성

## 이 노트북에서 하는 일
- 이 노트북은 연구 코호트를 정의하고 30일 이내 재입원 라벨을 생성한다.
- 대상 코호트: 첫 번째 입원(first admission)에 ICU 입원이 포함된 환자.
- 목표 라벨: 퇴원 후 30일 이내 재입원 여부.

## 이 노트북이 중요한 이유 (포트폴리오 / 면접 포인트)
임상 데이터 분석에서는 바로 모델링을 시작할 수 없다.
반드시 먼저 다음을 수행해야 한다:
1) 분석 대상이 되는 환자 집단을 정의하고 (코호트 정의),
2) 분석 결과(Outcome)를 데이터 관점에서 명확히 정의해야 한다 (라벨 생성).

이 노트북은 다음 역량을 보여준다:
- 관계형 임상 데이터 테이블에 대한 이해
- 시간 기반 결과 변수(Outcome)를 올바르게 처리하는 능력
- EDA 및 모델링을 위한 유효한 데이터셋을 구성하는 능력

# Step 1. Load required tables(데이터 불러오기 목적)

We load only the tables needed for cohort + label construction:

- PATIENTS: patient demographics (e.g., gender, date of birth)
- ADMISSIONS: hospital admissions (admit time, discharge time, etc.)
- ICUSTAYS: ICU stay records (used to filter ICU-related admissions)

Why these tables?
- Readmission is defined using **admission and discharge timestamps** (ADMISSIONS).
- We restrict the study to ICU patients using **ICUSTAYS**.
- We later create simple features like age and sex using **PATIENTS**.

---

# Step 1. 필요한 테이블 불러오기 (데이터 로드 목적)

코호트 정의와 라벨 생성을 위해 필요한 테이블만 선택적으로 로드한다.

- PATIENTS: 환자 인구통계 정보 (예: 성별, 생년월일)

- ADMISSIONS: 병원 입원 정보 (입원 시각, 퇴원 시각 등)

- ICUSTAYS: ICU 입원 기록 (ICU 입원이 포함된 입원 건을 필터링하는 데 사용)

왜 이 테이블들을 사용하는가?
- 재입원 여부는 입원·퇴원 시각 정보를 기준으로 정의되므로 ADMISSIONS가 필요하다.
- 연구 대상을 ICU 환자로 제한하기 위해 ICUSTAYS를 사용한다.
- 이후 나이, 성별과 같은 기본 특성(feature) 을 생성하기 위해 PATIENTS를 활용한다.

In [ ]:
import pandas as pd

# base_path는 Colab 런타임에서 clone된 레포 폴더 위치 기준으로 설정함
# (중요) data 폴더가 이 경로 아래에 있어야 CSV를 읽어올 수 있음.
base_path = "/content/mimic-readmission-analysis/data"

patients = pd.read_csv(f"{base_path}/PATIENTS.csv")
admissions = pd.read_csv(f"{base_path}/ADMISSIONS.csv")
icustays = pd.read_csv(f"{base_path}/ICUSTAYS.csv")

In [ ]:
# head()는 "컬럼이 제대로 읽혔는지" 빠르게 확인하는 용도
patients.head()

,row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
0,9467,10006,F,2094-03-05 00:00:00,2165-08-12 00:00:00,2165-08-12 00:00:00,2165-08-12 00:00:00,1
1,9472,10011,F,2090-06-05 00:00:00,2126-08-28 00:00:00,2126-08-28 00:00:00,NaN,1
2,9474,10013,F,2038-09-03 00:00:00,2125-10-07 00:00:00,2125-10-07 00:00:00,2125-10-07 00:00:00,1
3,9478,10017,F,2075-09-21 00:00:00,2152-09-12 00:00:00,NaN,2152-09-12 00:00:00,1
4,9479,10019,M,2114-06-20 00:00:00,2163-05-15 00:00:00,2163-05-15 00:00:00,2163-05-15 00:00:00,1


In [ ]:
# shape는 (행 개수, 열 개수)
# - admissions 행 개수가 너무 작거나 0이면 경로/파일 문제 가능성
# - icustays도 마찬가지
admissions.shape, icustays.shape

((129, 19), (136, 12))

# Step 2. Convert time columns to datetime(날짜를 datetime으로 바꾸는 이유)

Readmission is a **time-based label**.

We need to compute:
- the time from **discharge** to the **next admission**
- whether that time gap is **≤ 30 days**

If dates are stored as strings (object), we cannot compute time differences correctly.
Therefore we convert relevant columns to pandas datetime type.

Columns used:
- PATIENTS: dob (for age calculation later)
- ADMISSIONS: admittime, dischtime (for ordering admissions and label construction)

---

# Step 2. 시간 컬럼을 datetime 형식으로 변환 (날짜를 datetime으로 바꾸는 이유)

재입원 여부는 시간을 기준으로 정의되는 라벨이다.

이를 위해 다음 계산이 필요하다:
- 퇴원 시점부터 다음 입원까지의 시간 간격
- 해당 간격이 30일 이내인지 여부

날짜 정보가 문자열(object) 형태로 저장되어 있으면
시간 차이를 정확하게 계산할 수 없기 때문에,
관련 컬럼들을 pandas의 datetime 타입으로 변환한다.

사용되는 컬럼
- PATIENTS: dob (이후 나이 계산에 사용)
- ADMISSIONS: admittime, dischtime
(입원 순서 정렬 및 재입원 라벨 생성에 사용)

In [ ]:
# dob는 나이 계산을 위해 사용
patients['dob'] = pd.to_datetime(patients['dob'])

# admittime: 입원 시각
# dischtime: 퇴원 시각
# 이 둘이 datetime이어야 "퇴원 후 며칠 뒤 재입원" 같은 계산이 가능함.
admissions['admittime'] = pd.to_datetime(admissions['admittime'])
admissions['dischtime'] = pd.to_datetime(admissions['dischtime'])

In [ ]:
# dtypes로 실제 변환이 되었는지 확인
admissions[['admittime', 'dischtime']].dtypes

,0
admittime,datetime64[ns]
dischtime,datetime64[ns]


# Step 3. Create admission order per patient(환자별 입원 순서(admission_order) 만드는 이유)

A single patient can have multiple admissions.
To define "first admission" and "readmission", we must know:

- Which admission is the 1st, 2nd, 3rd... for each subject_id?

We will:
1) Sort admissions by (subject_id, admittime)
2) Assign admission_order:
   - 1 = first admission
   - 2+ = subsequent admissions

Why this matters:
- Our project focuses on **first-time admissions** (baseline admission)
- Readmission is defined relative to that baseline admission

---

# Step 3. 환자별 입원 순서(admission_order) 만들기
왜 필요한가?
한 환자(subject_id)는 여러 번 입원할 수 있음

- 그래서 각 입원이 1번째 입원인지 2번째, 3번째 입원인지 구분해야 함

무엇을 하려는가?
1) (subject_id, admittime) 기준으로 입원 기록 정렬
2) 같은 환자 안에서
  - admission_order = 1 → 첫 입원
  - admission_order = 2 이상 → 이후 입원

왜 중요한가?
- 이 프로젝트는 첫 입원(first admission) 을 기준(baseline)으로 분석함
- 재입원(readmission) 은 첫 입원 이후에 발생한 입원을 기준으로 정의됨

In [ ]:
# 환자별 + 시간순으로 입원 기록을 정렬해야 "다음 입원"이 의미를 가짐
admissions = admissions.sort_values(['subject_id', 'admittime'])

# groupby(subject_id)로 환자별 그룹을 만들고,
# cumcount()로 0,1,2,... 순서를 매긴 뒤 +1 해서 1부터 시작하게 만듦
admissions['admission_order'] = admissions.groupby('subject_id').cumcount() + 1

In [ ]:
# 결과 확인: 같은 subject_id 안에서 admission_order가 1,2,3... 증가하는지 체크
admissions[['subject_id', 'hadm_id', 'admittime', 'admission_order']].head()

,subject_id,hadm_id,admittime,admission_order
0,10006,142345,2164-10-23 21:09:00,1
1,10011,105331,2126-08-14 22:32:00,1
2,10013,165520,2125-10-04 23:36:00,1
3,10017,199207,2149-05-26 17:19:00,1
4,10019,177759,2163-05-14 20:43:00,1


# Step 4. Find the next admission time (next_admittime)(next_admittime 만드는 이유 - 재입원 라벨의 핵심)

To know if a patient was readmitted after an admission,
we need the timestamp of the **next** hospital admission.

For each patient:
- current admission time = admittime
- next admission time = admittime of the next row within the same patient group

We use groupby + shift(-1):
- shift(-1) moves the next row up, aligning it with the current admission.

Interpretation:
- If next_admittime exists → the patient had another admission later
- If next_admittime is NaT → this admission is the last admission in our data

---

# Step 4. 다음 입원 시각(next_admittime) 찾기(next_admittime 만드는 이유 – 재입원 라벨의 핵심)

재입원이 있었는지 판단하려면,
각 입원 이후에 다시 병원에 입원했는지를 알아야 한다.
이를 위해 다음 입원의 시각이 필요하다.

환자별로 해야 할 일
- 현재 입원 시각 = admittime
- 다음 입원 시각 = 같은 환자 안에서, 바로 다음 행의 admittime

사용하는 방법
- groupby(subject_id)로 환자별로 묶은 뒤 shift(-1) 사용

shift(-1)은 다음 행의 값을 위로 한 칸 당겨서, 현재 입원과 “다음 입원”을 같은 행에 맞춰준다.

해석:
- next_admittime 값이 존재하면 → 이 입원 이후에 다음 입원이 있었다 (재입원 가능)
- next_admittime가 NaT이면 → 이 입원은 데이터상 마지막 입원이다

In [ ]:
# shift(-1): 같은 환자 내에서 "다음 행"의 admittime을 현재 행으로 가져옴
admissions['next_admittime'] = admissions.groupby('subject_id')['admittime'].shift(-1)

In [ ]:
# 확인: last admission은 next_admittime이 NaT일 수 있음 (정상)
admissions[['subject_id', 'admittime', 'next_admittime']].head()

,subject_id,admittime,next_admittime
0,10006,2164-10-23 21:09:00,NaT
1,10011,2126-08-14 22:32:00,NaT
2,10013,2125-10-04 23:36:00,NaT
3,10017,2149-05-26 17:19:00,NaT
4,10019,2163-05-14 20:43:00,NaT


# Step 5. Calculate days to next admission(days_to_next_admit 계산 이유)

Readmission is defined by the time gap:
- (next admission time) - (discharge time)

We compute:
days_to_next_admit = next_admittime - dischtime

Important notes:
- We use dischtime (discharge), not admittime, because "readmission" usually means
  returning to the hospital **after discharge**.
- If next_admittime is NaT, the difference becomes NaN → meaning no next admission.

---

# Step 5. 다음 입원까지 걸린 일수(days_to_next_admit) 계산(days_to_next_admit 계산 이유)

재입원은 시간 간격(time gap) 으로 정의된다.
- 즉, 다음 입원 시각 − 현재 입원의 퇴원 시각을 계산해야 한다.

계산식:
days_to_next_admit = next_admittime - dischtime

중요한 점:
- admittime이 아니라 dischtime(퇴원 시각) 을 기준으로 사용한다
→ 재입원은 퇴원 후 다시 병원에 돌아오는 것을 의미하기 때문
- next_admittime가 NaT이면
→ 계산 결과도 NaN
→ 즉, 다음 입원이 없음을 의미

In [ ]:
# timedelta(시간 차이)를 만든 뒤 .dt.days로 '일(day)' 단위 정수로 변환
admissions['days_to_next_admit'] = (admissions['next_admittime'] - admissions['dischtime']).dt.days

In [ ]:
# 값 확인
admissions[['days_to_next_admit']].head()

,days_to_next_admit
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN


# Step 6. Create 30-day readmission label (readmitted_30d)(readmitted_30d 라벨 만드는 이유)

Outcome definition:
- readmitted_30d = 1 if the next admission occurs within 30 days after discharge
- readmitted_30d = 0 otherwise

Why 30 days?
- 30-day readmission is a common hospital quality metric.
- It is widely used in clinical research and healthcare operations.

Implementation:
- If days_to_next_admit <= 30 → label = 1
- If days_to_next_admit > 30 or missing → label = 0

Note:
- Missing (NaN) means no next admission in the dataset.
  We treat it as not readmitted within 30 days (0) for this dataset.

---
# Step 6. 30일 이내 재입원 라벨(readmitted_30d) 생성(readmitted_30d 라벨을 만드는 이유)

결과 변수 정의 (Outcome definition)
- readmitted_30d = 1
→ 퇴원 후 30일 이내에 다음 입원이 발생한 경우
- readmitted_30d = 0
→ 그 외의 경우

왜 30일인가?
- 30일 재입원률은 병원 질 평가에서 널리 쓰이는 지표
- 임상 연구와 의료 운영(quality metric)에서 표준적으로 사용됨

구현 기준
- days_to_next_admit <= 30
→ readmitted_30d = 1
- days_to_next_admit > 30 또는 값이 없음
→ readmitted_30d = 0

주의사항 (Note)
- days_to_next_admit가 NaN인 경우
→ 데이터상 다음 입원이 없음
- 이 데이터셋에서는 이를
→ 30일 이내 재입원하지 않음(0) 으로 처리함

In [ ]:
# True/False를 1/0으로 변환하기 위해 astype(int) 사용
admissions['readmitted_30d'] = (admissions['days_to_next_admit'] <= 30).astype(int)

In [ ]:
# 라벨 분포 확인: 0과 1이 얼마나 있는지
admissions['readmitted_30d'].value_counts()

,count
readmitted_30d,
0,118
1,11


# Step 7. Restrict to first admission per patient (baseline admission)("첫 입원만" 고르는 이유)

Our project focuses on "first admission information" as baseline.

So we keep only:
- admission_order == 1

This ensures:
- each patient contributes exactly one baseline admission record
- readmission label is defined relative to that baseline
- the dataset becomes a patient-level cohort suitable for EDA/modelling

---

# Step 7. 환자별 첫 입원만 선택 (baseline admission)(“첫 입원만” 고르는 이유)

이 프로젝트는 환자의 ‘첫 입원 정보’를 기준(baseline) 으로 분석한다.

그래서 다음 조건만 남긴다:
- admission_order == 1

이렇게 하면 보장되는 것:
- 각 환자는 정확히 한 번만 데이터에 포함됨
- 재입원 라벨은 첫 입원 이후를 기준으로 정의됨
- 데이터셋이
→ 환자 단위(patient-level) cohort가 되어
→ EDA 및 모델링에 적합해짐

In [ ]:
first_admissions = admissions[admissions['admission_order'] == 1].copy()

In [ ]:
# baseline admission에서 30일 재입원 라벨 분포 확인
first_admissions['readmitted_30d'].value_counts()

,count
readmitted_30d,
0,92
1,8


# Step 8. Restrict cohort to ICU-related first admissions(ICU환자만 필터링하는 이유)

The project scope is ICU patients.

How do we identify ICU admissions?
- ICUSTAYS contains ICU stay records, including hadm_id.
- If an admission (hadm_id) appears in ICUSTAYS, it means that hospital admission
  included an ICU stay.

Therefore:
1) Extract all ICU hadm_id values from ICUSTAYS
2) Keep only baseline admissions whose hadm_id is in that ICU list

This yields:
- first_icu_admissions = first admission + ICU stay cohort

---

# Step 8. ICU 관련 첫 입원만 선택 (ICU 환자 cohort)(ICU 환자만 필터링하는 이유)

이 프로젝트의 분석 대상은 ICU 환자이다.

ICU 입원을 어떻게 구분하나?
- ICUSTAYS 테이블에는 ICU 입원 기록이 저장되어 있고,
각 ICU stay는 hadm_id(병원 입원 ID)와 연결되어 있다.
- 즉, 어떤 입원(hadm_id)이 ICUSTAYS에 등장한다면
→ 그 입원 기간 중 ICU에 입실한 적이 있었다는 의미이다.

적용 방법:
1) ICUSTAYS 테이블에서
→ ICU에 해당하는 모든 hadm_id를 추출
2) baseline admission 중에서
→ hadm_id가 이 ICU 목록에 포함된 경우만 유지

결과:
- first_icu_admissions
→ 첫 입원 + ICU stay를 포함한 환자 cohort

In [ ]:
# ICU admission(hadm_id) 목록을 추출
icu_hadm_ids = icustays['hadm_id'].unique()

# baseline admissions 중 ICU hadm_id에 포함되는 것만 남김
first_icu_admissions = first_admissions[first_admissions['hadm_id'].isin(icu_hadm_ids)].copy()

In [ ]:
# 최종 cohort 크기 확인
first_icu_admissions.shape

(100, 23)

# Step 9. Merge patient demographics (age, gender)(왜 age / gender를 여기서 만드는가)

After defining the cohort and outcome label, we add **baseline demographic variables**.

Why do this in the cohort definition step?
- Age and gender are **baseline patient characteristics**
- They describe "who the patients are" before any analysis
- EDA and modelling should not re-calculate these repeatedly

In clinical studies, age and sex are typically reported as part of the
"study population characteristics", not as derived variables in EDA.

Data source:
- PATIENTS table provides:
  - subject_id (patient identifier)
  - gender
  - dob (date of birth)

---
# Step 9. 환자 기본 인구학적 정보(age, gender) 병합(왜 age / gender를 여기서 만드는가)

cohort와 outcome 라벨을 정의한 뒤,
기본 환자 특성(baseline demographic variables) 을 추가한다.

왜 cohort 정의 단계에서 하는가?
- 나이와 성별은 환자의 기본 특성
- 분석 이전에
→ “어떤 환자들인가”를 설명하는 변수들
- EDA나 모델링 단계에서
→ 매번 다시 계산할 필요가 없음

임상 연구에서는
→ 나이·성별은 EDA에서 파생된 값이 아니라
→ 연구 대상자 특성(study population characteristics) 으로 보고됨

데이터 출처:
- PATIENTS 테이블에서 제공:
  - subject_id : 환자 식별자
  - gender : 성별
  - dob : 생년월일

In [ ]:
# PATIENTS 테이블에서 필요한 컬럼만 선택
patients_demo = patients[['subject_id', 'gender', 'dob']]

# subject_id 기준으로 병합
# how='left' 의미:
# - cohort(first_icu_admissions)는 그대로 유지
# - 환자 정보가 있으면 붙이고, 없으면 NaN
first_icu_admissions = first_icu_admissions.merge(
    patients_demo,
    on='subject_id',
    how='left'
)

## Age calculation(나이(age) 계산 방법 설명)

Age is calculated as the difference between:
- admission time (admittime)
- date of birth (dob)

Why use admittime?
- Age should represent the patient's age **at the time of hospital admission**
- This matches how age is defined in most clinical studies

Note on MIMIC data:
- In the demo dataset, DOB values are shifted for privacy
- The absolute age values may not be realistic
- However, **relative age differences are still valid** for analysis practice

---
## Age calculation (나이 계산 방법)

나이는 다음 두 시점의 차이로 계산한다:
- 입원 시각 (admittime)
- 생년월일 (dob)

왜 admittime을 기준으로 하나?
- 나이는 병원에 입원했을 당시의 나이를 의미해야 함
- 이는 대부분의 임상 연구에서 사용하는 나이 정의와 동일함

MIMIC 데이터 관련 주의사항:
- 데모 데이터셋에서는 개인정보 보호를 위해
→ dob 값이 임의로 이동(shift) 되어 있음
- 따라서 절대적인 나이 값은 현실과 다를 수 있음
- 하지만 환자 간 상대적인 나이 차이에 따른 패턴 분석
은 연습용 분석에서는 유효함

In [ ]:
# 나이 계산: (입원 연도 - 출생 연도)
# 간단한 연 단위 계산으로 충분 (demo 데이터 & 교육 목적)
first_icu_admissions['age'] = (
    first_icu_admissions['admittime'].dt.year -
    first_icu_admissions['dob'].dt.year
)

In [ ]:
first_icu_admissions[['age', 'gender']].head()

,age,gender
0,70,F
1,36,F
2,87,F
3,74,F
4,49,M


In [ ]:
# 나이 분포 간단 확인
first_icu_admissions['age'].describe()

,age
count,100.000000
mean,88.400000
std,64.836858
min,17.000000
25%,64.750000
50%,76.500000
75%,86.000000
max,300.000000


# Step 10. Calculate ICU length of stay (ICU LOS)(ICU LOS를 만드는 이유)

ICU length of stay (LOS) is a clinically meaningful variable.

Why ICU LOS matters:
- Longer ICU stay often indicates higher severity or complications
- Patients with prolonged ICU stays may have higher readmission risk
- ICU LOS is frequently used in clinical outcome studies

Data source:
- ICUSTAYS table provides:
  - hadm_id (hospital admission ID)
  - intime (ICU admission time)
  - outtime (ICU discharge time)

Important consideration:
- A single hospital admission (hadm_id) can have **multiple ICU stays**
- Therefore, we aggregate ICU LOS **per hospital admission**

---
# Step 10. ICU 재원 기간(ICU LOS) 계산(ICU LOS를 만드는 이유)

ICU 재원 기간(ICU length of stay, LOS)은
임상적으로 의미 있는 중요한 변수이다.

왜 ICU LOS가 중요한가?:
- ICU에 오래 머물수록
→ 질병 중증도가 높거나 합병증이 있었을 가능성이 큼
- ICU 재원 기간이 긴 환자는
→ 재입원 위험이 더 높을 수 있음
- ICU LOS는
→ 임상 결과(outcome) 연구에서 자주 사용되는 변수

데이터 출처:
- ICUSTAYS 테이블에서 제공:
  - hadm_id : 병원 입원 ID
  - intime : ICU 입실 시각
  - outtime : ICU 퇴실 시각

중요한 고려사항:
- 하나의 병원 입원(hadm_id) 안에서
→ ICU 입실이 여러 번 발생할 수 있음
- 따라서
→ ICU LOS는 병원 입원 단위(hadm_id)로 합산(aggregate) 해야 함

In [ ]:
# ICU 재원 기간 계산 (시간 단위 → 일 단위)
icustays['intime'] = pd.to_datetime(icustays['intime'])
icustays['outtime'] = pd.to_datetime(icustays['outtime'])

# 각 ICU stay별 LOS (일 단위)
icustays['icu_los'] = (
    icustays['outtime'] - icustays['intime']
).dt.total_seconds() / (60 * 60 * 24)

## Aggregating ICU LOS per admission

Since one admission can include multiple ICU stays:
- We sum ICU LOS across all ICU stays within the same hadm_id

This gives:
- total ICU length of stay per hospital admission

---
## ICU LOS를 입원 단위로 합산 (Aggregating ICU LOS per admission)
하나의 병원 입원(hadm_id)에는
여러 번의 ICU 입실이 포함될 수 있다.
- 따라서 같은 hadm_id에 속한 모든 ICU stay의 ICU LOS를 합산한다.

- 병원 입원 1건(hadm_id)당
→ 총 ICU 재원 기간(total ICU length of stay) 이 계산된다.

In [ ]:
# hadm_id 기준으로 ICU LOS 합산
icu_los_per_admission = (
    icustays
    .groupby('hadm_id')['icu_los']
    .sum()
    .reset_index()
)

In [ ]:
# ICU LOS를 cohort 테이블에 병합
first_icu_admissions = first_icu_admissions.merge(
    icu_los_per_admission,
    on='hadm_id',
    how='left'
)

In [ ]:
first_icu_admissions[['icu_los']].describe()

,icu_los
count,100.000000
mean,4.771405
std,6.776915
min,0.105926
25%,1.230729
50%,2.259190
75%,4.618220
max,35.406516


# Final cohort produced in this notebook

The final dataset `first_icu_admissions` represents:

- One baseline (first) hospital admission per patient
- Admissions that included at least one ICU stay
- A binary outcome indicating 30-day readmission
- Baseline demographic variables (age, gender)
- ICU length of stay (icu_los)

This dataset is ready for:
- Exploratory data analysis (03_readmission_eda.ipynb)
- Baseline predictive modelling (04_baseline_model.ipynb)

---
# 이 노트북에서 생성된 최종 cohort
최종 데이터셋 first_icu_admissions는 다음을 의미한다:

- 환자당 하나의 기준 입원(baseline, 첫 입원) 만 포함
- ICU 입실이 최소 1회 이상 포함된 입원만 선택
- 30일 이내 재입원 여부를 나타내는 이진 결과 변수 포함
- 기본 인구학적 변수 포함 (나이, 성별)
- ICU 재원 기간(ICU LOS) 포함

이 데이터셋으로 할 수 있는 것:
- 탐색적 데이터 분석(EDA)
→ 03_readmission_eda.ipynb
- 기본 예측 모델링
→ 04_baseline_model.ipynb

In [ ]:
first_icu_admissions.columns

Index(['row_id', 'subject_id', 'hadm_id', 'admittime', 'dischtime',
       'deathtime', 'admission_type', 'admission_location',
       'discharge_location', 'insurance', 'language', 'religion',
       'marital_status', 'ethnicity', 'edregtime', 'edouttime', 'diagnosis',
       'hospital_expire_flag', 'has_chartevents_data', 'admission_order',
       'next_admittime', 'days_to_next_admit', 'readmitted_30d'],
      dtype='object')

In [ ]:
first_icu_admissions.shape

(100, 27)

In [ ]:
# Save final cohort for downstream analysis
first_icu_admissions.to_csv(
    "/content/mimic-readmission-analysis/data/first_icu_admissions.csv",
    index=False
)